In [29]:
!pip3 freeze

alembic==1.7.7
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
async-generator==1.10
attrs==22.2.0
Babel==2.5.1
backcall==0.2.0
bleach==4.1.0
certipy==0.1.3
cffi==1.15.1
chardet==3.0.4
cryptography==40.0.2
dataclasses==0.8
dbus-python==1.2.4
decorator==4.2.1
defusedxml==0.7.1
dnspython==1.15.0
entrypoints==0.4
file-magic==0.3.0
gpg==1.13.1
greenlet==2.0.2
gssapi==1.5.1
html5lib==0.999999999
idna==2.5
importlib-metadata==4.8.3
importlib-resources==5.4.0
iotop==0.6
ipaclient==4.9.11
ipalib==4.9.11
ipaplatform==4.9.11
ipapython==4.9.11
ipykernel==5.5.6
ipython==7.16.3
ipython-genutils==0.2.0
isc==2.0
jedi==0.17.2
jhub-remote-user-authenticator==0.1.0
Jinja2==3.0.3
jsonschema==3.2.0
jupyter-client==7.1.2
jupyter-core==4.9.2
jupyter-telemetry==0.1.0
jupyterhub==2.3.1
jupyterlab-pygments==0.1.2
jwcrypto==0.5.0
libcomps==0.1.18
lxml==4.2.3
Mako==1.1.6
MarkupSafe==2.0.1
mistune==0.8.4
nbclient==0.5.9
nbconvert==6.0.7
nbformat==5.1.3
nest-asyncio==1.5.8
netaddr==0.7.19
netifaces==0.10.6
nftabl

In [34]:
!pip3 install pyocto pyarrow

  Could not find a version that satisfies the requirement pyocto (from versions: )
No matching distribution found for pyocto


In [4]:
import pyocto
import pandas as pd
import datetime
import matplotlib.pyplot as plt

In [25]:
picks = pd.read_csv("~/notebooks/data/phase_association/pick")
stations = pd.read_csv("~/notebooks/data/phase_association/stations")

In [26]:
velocity_model = pyocto.VelocityModel0D(
    p_velocity=7.0,
    s_velocity=4.0,
    tolerance=2.0,
)

AttributeError: module 'pyocto' has no attribute 'VelocityModel0D'

In [5]:
associator = pyocto.OctoAssociator.from_area(
    lat=(42, 50),
    lon=(-126, -123),
    zlim=(0, 200),
    time_before=300,
    velocity_model=velocity_model,
    n_picks=10,
    n_p_and_s_picks=4,
)

NameError: name 'velocity_model' is not defined